In [ ]:
#Смотрим файлы в cd
import os
import pandas as pd
import folium
import geopandas as gpd
files = os.listdir()
files

['.config', 'Kapali_Mahalleler.xlsx', 'sample_data']

## Чтение файла и создание столбца с полным названием

In [ ]:
# Читаем файл с таблицей
df = pd.read_excel('Kapali_Mahalleler.xlsx', index_col=0)

In [ ]:
# создаем словарь с полными именами
full_names = []
for x in range(len(df)):
  name = ', '.join(df.iloc[x])
  full_names.append(name)

In [ ]:
# новый столбик
df['Full Name'] = full_names

## Работа с названиями

In [ ]:
# удаляем слово район
df['Full Name'] = df['Full Name'].str.replace('MAHALLESİ', '')

In [ ]:
#@title Удалить все после слова KÖYÜ для улчушения поиска
# Необходимо удалить все после слова KÖYÜ для улчушения поиска
#df[df['IL'].isin(['Adıyaman'])]['Full Name'].value_counts()
df['Full Name'] = df['Full Name'].str.replace(r'(KÖYÜ).*', r'\1', regex=True)

In [ ]:
#@title Удалить слова TÜM MAHALLELER для улчушения поиска
# Необходимо удалить слова TÜM MAHALLELER для улчушения поиска
#df[df['MAHALLE'].isin(['TÜM MAHALLELER'])]['Full Name'].value_counts()
df['Full Name'] = df['Full Name'].str.replace('TÜM MAHALLELER', '')

In [ ]:
#@title Все города
# Установите максимальное количество строк для отображения
pd.set_option('display.max_rows', None)

# Предположим, у вас есть DataFrame df
# Вывод всех значений и их количества в столбце 'IL'
print(df['IL'].value_counts(dropna=False))


IL
Şanlıurfa         170
Gaziantep         162
Hatay             109
Kilis              83
Adana              75
Mersin             63
Mardin             59
İstanbul           54
Kahramanmaraş      32
İzmir              27
Afyonkarahisar     20
Adıyaman           19
Bursa              19
Konya              18
Ankara             17
Malatya            16
Nevşehir           15
Kayseri            14
Elazığ             11
Aksaray            11
Niğde              10
Burdur             10
Çanakkale          10
Bolu               10
Antalya            10
Çankırı             9
Osmaniye            8
Yalova              7
Kastamonu           7
Karabük             7
Çorum               7
Uşak                6
Sivas               5
Samsun              5
Erzincan            5
Isparta             4
Bilecik             4
Yozgat              4
Kırşehir            3
Eskişehir           3
Sakarya             3
Bartın              3
Diyarbakır          3
Düzce               2
Batman              2
Muğla  

In [ ]:
# Вернуть настройки обратно к значению по умолчанию
pd.reset_option('display.max_rows')


In [ ]:
#@title Поменять
# İstanbul, Küçükçekmece, KÜÇÜKÇEKMECE İKİTELLİOSB	меняем на İstanbul, Küçükçekmece, İKİTELLİ
df['Full Name'] = df['Full Name'].str.replace('İstanbul, Küçükçekmece, KÜÇÜKÇEKMECE İKİTELLİOSB', 'İstanbul, Küçükçekmece, İKİTELLİ')

## Создание координат

In [ ]:
# Долгота
#gdf.geometry.iloc[0].x

35.220982

In [ ]:
# Широта
#gdf.geometry.iloc[0].y

37.106358

In [ ]:
shirota = []
dolgota = []
for x in range(len(df)):
  try:
      gdf = gpd.tools.geocode(df.iloc[x, 3], timeout=10)
      if not gdf.empty:
          dolgota.append(gdf.geometry.iloc[0].x)
          shirota.append(gdf.geometry.iloc[0].y)
      else:
          dolgota.append('Uncalculated')
          shirota.append('Uncalculated')
  except Exception as e:
      print(f"Error geocoding {df.iloc[x, 3]}: {e}")
      dolgota.append('Uncalculated')
      shirota.append('Uncalculated')



Error geocoding Aksaray, Aksaray Merkez, TAŞPINAR BELDESİ, ERENLER OSB : UnsupportedOperationException: getX called on empty Point

Error geocoding Bayburt, Bayburt Merkez, YOLALTI KÖYÜ: UnsupportedOperationException: getX called on empty Point

Error geocoding Bilecik, Bilecik Merkez, BAŞKÖY KÖYÜ: UnsupportedOperationException: getX called on empty Point

Error geocoding Gümüşhane, Gümüşhane Merkez, YAĞLIDERE KÖYÜ: UnsupportedOperationException: getX called on empty Point

Error geocoding Kilis, Elbeyli, YALÇIN NANE : UnsupportedOperationException: getX called on empty Point



In [ ]:
df['shirota'] = shirota
df['dolgota'] = dolgota

In [ ]:
df.loc[(df['shirota'] == 'Uncalculated')]

,IL,ILCE,MAHALLE,Full Name,shirota,dolgota
Sıra No,,,,,,
123,Aksaray,Aksaray Merkez,"TAŞPINAR BELDESİ, ERENLER OSB MAHALLESİ","Aksaray, Aksaray Merkez, TAŞPINAR BELDESİ, ERE...",Uncalculated,Uncalculated
160,Bayburt,Bayburt Merkez,"YOLALTI KÖYÜ, YOLALTI OSB MEVKİİ","Bayburt, Bayburt Merkez, YOLALTI KÖYÜ",Uncalculated,Uncalculated
161,Bilecik,Bilecik Merkez,"BAŞKÖY KÖYÜ, MÜCAVİR DIŞI MEVKİİ","Bilecik, Bilecik Merkez, BAŞKÖY KÖYÜ",Uncalculated,Uncalculated
419,Gümüşhane,Gümüşhane Merkez,"YAĞLIDERE KÖYÜ, BULUT MEVKİİ","Gümüşhane, Gümüşhane Merkez, YAĞLIDERE KÖYÜ",Uncalculated,Uncalculated
686,Kilis,Elbeyli,YALÇIN NANE MAHALLESİ,"Kilis, Elbeyli, YALÇIN NANE",Uncalculated,Uncalculated


In [ ]:
# Исправляем оставшиеся ошибки
new_names = ['Aksaray, Aksaray Merkez, TAŞPINAR', 'Bayburt, Bayburt Merkez, YOLALTI', 'Bilecik, Bilecik Merkez, BAŞKÖY', 'Gümüşhane, Gümüşhane Merkez, YAĞLIDERE', 'Kilis, Elbeyli']
df.loc[(df['shirota'] == 'Uncalculated'), 'Full Name'] = new_names

In [ ]:
df.loc[(df['shirota'] == 'Uncalculated')]

,IL,ILCE,MAHALLE,Full Name,shirota,dolgota
Sıra No,,,,,,
123,Aksaray,Aksaray Merkez,"TAŞPINAR BELDESİ, ERENLER OSB MAHALLESİ","Aksaray, Aksaray Merkez, TAŞPINAR",Uncalculated,Uncalculated
160,Bayburt,Bayburt Merkez,"YOLALTI KÖYÜ, YOLALTI OSB MEVKİİ","Bayburt, Bayburt Merkez, YOLALTI",Uncalculated,Uncalculated
161,Bilecik,Bilecik Merkez,"BAŞKÖY KÖYÜ, MÜCAVİR DIŞI MEVKİİ","Bilecik, Bilecik Merkez, BAŞKÖY",Uncalculated,Uncalculated
419,Gümüşhane,Gümüşhane Merkez,"YAĞLIDERE KÖYÜ, BULUT MEVKİİ","Gümüşhane, Gümüşhane Merkez, YAĞLIDERE",Uncalculated,Uncalculated
686,Kilis,Elbeyli,YALÇIN NANE MAHALLESİ,"Kilis, Elbeyli",Uncalculated,Uncalculated


In [ ]:
shirota = []
dolgota = []
for x in range(len(df.loc[(df['shirota'] == 'Uncalculated')] )):
  try:
      gdf = gpd.tools.geocode(df.loc[(df['shirota'] == 'Uncalculated')].iloc[x, 3], timeout=10)
      if not gdf.empty:
          dolgota.append(gdf.geometry.iloc[0].x)
          shirota.append(gdf.geometry.iloc[0].y)
      else:
          dolgota.append('Uncalculated')
          shirota.append('Uncalculated')
  except Exception as e:
      print(f"Error geocoding {df.loc[(df['shirota'] == 'Uncalculated')].iloc[x, 3]}: {e}")
      dolgota.append('Uncalculated')
      shirota.append('Uncalculated')



In [ ]:
df.loc[(df['shirota'] == 'Uncalculated'), 'shirota'] = shirota
df.loc[(df['dolgota'] == 'Uncalculated'), 'dolgota'] = dolgota

In [ ]:
df['shirota'].iloc[0]

37.0383318

In [ ]:
pd.set_option('display.max_rows', None)
df.loc[(df['IL']=='Kilis') & (df['ILCE']=='Kilis Merkez')]

,IL,ILCE,MAHALLE,Full Name,shirota,dolgota
Sıra No,,,,,,
689,Kilis,Kilis Merkez,BÜYÜKKÜTAH MAHALLESİ,"Kilis, Kilis Merkez, BÜYÜKKÜTAH",36.7154,37.116994
690,Kilis,Kilis Merkez,TEKYE MAHALLESİ,"Kilis, Kilis Merkez, TEKYE",36.715447,37.113378
691,Kilis,Kilis Merkez,NURETTİN MAHALLESİ,"Kilis, Kilis Merkez, NURETTİN",36.713433,37.115761
692,Kilis,Kilis Merkez,KETENCİLER MAHALLESİ,"Kilis, Kilis Merkez, KETENCİLER",36.717014,37.115223
693,Kilis,Kilis Merkez,CANPOLAT PAŞA MAHALLESİ,"Kilis, Kilis Merkez, CANPOLAT PAŞA",36.714482,37.115419
694,Kilis,Kilis Merkez,TIRIKLI MAHALLESİ,"Kilis, Kilis Merkez, TIRIKLI",36.71785,37.115694
695,Kilis,Kilis Merkez,ALTINÜZÜM MAHALLESİ,"Kilis, Kilis Merkez, ALTINÜZÜM",39.929961,38.195614
696,Kilis,Kilis Merkez,YEDİ ARALIK MAHALLESİ,"Kilis, Kilis Merkez, YEDİ ARALIK",36.717284,37.112281
697,Kilis,Kilis Merkez,ŞIHLAR MAHALLESİ,"Kilis, Kilis Merkez, ŞIHLAR",36.715447,37.113378


In [ ]:
# Вернуть настройки обратно к значению по умолчанию
pd.reset_option('display.max_rows')

In [ ]:
# фикс точки на Крити
df.loc[
    (df['IL'] == 'Kilis') &
    (df['ILCE'] == 'Kilis Merkez') &
    (df['MAHALLE'] == 'CUMHURİYET MAHALLESİ'),
    ['shirota', 'dolgota']
] = [36.7126767, 37.1088736]


## Визуализация

In [ ]:
import folium

# Создаем пустую карту с произвольным центром
m = folium.Map(location=[39.0, 35.0], zoom_start=6)
for _, row in df.iterrows():
    folium.Marker([row['shirota'], row['dolgota']], popup=row['Full Name']).add_to(m)



In [ ]:
m

In [ ]:
# Только геометрия
import osmnx as ox
df['geometry'] = None

# Получаем полигоны
for idx, row in df.iterrows():
    try:
        gdf = ox.geocode_to_gdf(row['Full Name'])
    except Exception as e:
        print(f"[{idx}] Ошибка получения геометрии для '{row['Full Name']}': {e}")
        continue

    try:
        geom = gdf.loc[0, 'geometry']
    except Exception as e:
        print(f"[{idx}] Ошибка доступа к геометрии для '{row['Full Name']}': {e}")
        continue

    try:
        if geom.geom_type in ['Polygon', 'MultiPolygon']:
            df.at[idx, 'geometry'] = geom
        else:
            print(f"[{idx}] Геометрия не является полигоном: {geom.geom_type}")
    except Exception as e:
        print(f"[{idx}] Ошибка обработки геометрии для '{row['Full Name']}': {e}")


[2] Ошибка получения геометрии для 'Adana, Çukurova, MEMİŞLİ ': Nominatim did not geocode query 'Adana, Çukurova, MEMİŞLİ ' to a geometry of type (Multi)Polygon.
[3] Ошибка получения геометрии для 'Adana, Çukurova, FADIL ': Nominatim did not geocode query 'Adana, Çukurova, FADIL ' to a geometry of type (Multi)Polygon.
[4] Ошибка получения геометрии для 'Adana, Çukurova, BOZCALAR ': Nominatim did not geocode query 'Adana, Çukurova, BOZCALAR ' to a geometry of type (Multi)Polygon.
[5] Ошибка получения геометрии для 'Adana, Çukurova, ÖRCÜN ': Nominatim did not geocode query 'Adana, Çukurova, ÖRCÜN ' to a geometry of type (Multi)Polygon.
[6] Ошибка получения геометрии для 'Adana, Çukurova, KÜÇÜKÇINAR ': Nominatim did not geocode query 'Adana, Çukurova, KÜÇÜKÇINAR ' to a geometry of type (Multi)Polygon.
[7] Ошибка получения геометрии для 'Adana, Karataş, TABAKLAR ': Nominatim did not geocode query 'Adana, Karataş, TABAKLAR ' to a geometry of type (Multi)Polygon.
[8] Ошибка получения геометр

In [ ]:
df.iloc[20:40,]

,IL,ILCE,MAHALLE,Full Name,shirota,dolgota,poly_lat,poly_lon,geometry
Sıra No,,,,,,,,,
21,Adana,Seyhan,KOCAVEZİR MAHALLESİ,"Adana, Seyhan, KOCAVEZİR",36.984489,35.321603,36.984695,35.321095,"POLYGON ((35.3174883 36.9859158, 35.3174964 36..."
22,Adana,Seyhan,TEPEBAĞ MAHALLESİ,"Adana, Seyhan, TEPEBAĞ",36.98802,35.327482,36.988186,35.327374,"POLYGON ((35.3249931 36.9867942, 35.3250897 36..."
23,Adana,Seyhan,MİRZAÇELEBİ MAHALLESİ,"Adana, Seyhan, MİRZAÇELEBİ",36.981746,35.316324,36.981506,35.316093,"POLYGON ((35.3130519 36.9813586, 35.3130546 36..."
24,Adana,Seyhan,KURUKÖPRÜ MAHALLESİ,"Adana, Seyhan, KURUKÖPRÜ",36.988397,35.323276,36.988494,35.323376,"POLYGON ((35.3203851 36.9878783, 35.3209484 36..."
25,Adana,Seyhan,ULUCAMİİ MAHALLESİ,"Adana, Seyhan, ULUCAMİİ",36.985243,35.331817,36.985522,35.332217,"POLYGON ((35.3297165 36.9862736, 35.3298989 36..."
26,Adana,Seyhan,ALİDEDE MAHALLESİ,"Adana, Seyhan, ALİDEDE",36.981884,35.326195,36.982108,35.326466,"POLYGON ((35.3247946 36.9807308, 35.3250682 36..."
27,Adana,Seyhan,KARASOKU MAHALLESİ,"Adana, Seyhan, KARASOKU",36.985312,35.327353,36.985107,35.32728,"POLYGON ((35.3248483 36.983529, 35.3258407 36...."
28,Adana,Seyhan,GÜLPINAR MAHALLESİ,"Adana, Seyhan, GÜLPINAR",36.981026,35.309715,36.981045,35.309616,"POLYGON ((35.3057277 36.9833673, 35.3058609 36..."
29,Adana,Seyhan,HANEDAN MAHALLESİ,"Adana, Seyhan, HANEDAN",36.98706,35.320101,36.986949,35.319916,"POLYGON ((35.3174883 36.9859158, 35.3178853 36..."


In [ ]:
# Центр карты по средней координате
avg_lat = df['shirota'].dropna().astype(float).mean()
avg_lon = df['dolgota'].dropna().astype(float).mean()
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# Добавляем полигоны и маркеры
for idx, row in df.iterrows():
    name = row['Full Name']
    geom = row['geometry']
    lat = row['shirota']
    lon = row['dolgota']

    if geom is not None:
        try:
            gj = folium.GeoJson(data=geom, name=name)
            popup = folium.Popup(name, parse_html=True)
            gj.add_child(popup)
            gj.add_to(m)
        except Exception as e:
            print(f"[{idx}] Ошибка добавления полигона для '{name}': {e}")
    else:
        try:
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(name, parse_html=True),
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(m)
        except Exception as e:
            print(f"[{idx}] Ошибка добавления маркера для '{name}': {e}")

# Сохраняем карту
m.save("map_with_polygons_and_markers.html")
print("Карта сохранена в map_with_polygons_and_markers.html")

Карта сохранена в map_with_polygons_and_markers.html
